# Taxi Networks simulation

In this notebook, we will demonstrate code that simulates the behavior of taxis
in queueing networks. We are modeling this using a Jackson network, which models
taxis as customers traveling in the network. Taxis queue at each station $$i$$,
and are served by customers arriving at a rate of $$\mu_i$$. After a customer
gets on a taxi, it travels to another station $$j$$ with probability
$$r_{ij}$$. We assume the taxis 

In [26]:
import numpy as np
import scipy.stats as sps
from matplotlib import pyplot as plt
%matplotlib inline

Here we define the main objects we use in the simulation

In [97]:
class Node:
    '''A Node object in a Jackson network simulates a queue with an exponential service time.'''
    def __init__(self, node_id, lam, r, dt=1):
        '''
        node_id: Unique integer id for each node
        lam    : Service rate (vehicles per time)
        r      : Routing probability from node id to probability
        dt     : Timestep 
        '''
        assert sum(r.values()) == 1, 'Routing probabilities do not sum to 1!'
        self.id = node_id
        self.r = r
        self.lam = lam
        self.dt = dt
        self.n = 0  # Number in queue
        self.buffer = 0 # Buffer time

    def route_to(self):
        r = np.random.sample()
        s = 0
        for id, prob in self.r.items():
            s += prob
            if r < s:
                return id
                
    def step(self):
        # No customers in this period
        if self.buffer > self.dt:
            self.buffer -= self.dt
            return []

        # Use up buffer and add one customer, this produces the effect of having
        # the first taxi being served at time = 0
        time_spent = self.buffer
        num_samples = 1

        # Sample from exponential distribution
        while time_spent <= self.dt:
            time_spent += sps.exponential(1.0 / self._lambda)
            num_samples += 1
        num_samples -= 1
        self.buffer = time_spent - self.dt
        
        return [self.route_to() for _ in range(num_samples)]

    def step(self):
        num_samples = np.random.poisson(lam=self.lam * self.dt)
        return [self.route_to() for _ in range(num_samples)]

1.9875

2